In [1]:
import warnings
from PIL import Image
from utils import *
warnings.filterwarnings('ignore')

# elec with 결측치 처리(0)

In [2]:
df = pd.read_csv('../temp/dataset_02[시간별목적별전력온습도].csv')

df['base'] = df['use'] / df['sum']
df['sum'] = df.pop('sum')
# df.fillna(0, inplace=True)
cols = list(df.columns[7:-2])

for col in cols:
    df[col] *= df['base']

elec = df.drop(['use', 'temp', 'humid', 'base', 'sum'], axis=1)
elec

,year,month,day,hour,공용공간,학생자율,교수연구,연구공간,학술지원,행정지원,강의지원
0,2017,1,1,1,2.422317,2.422317,2.971376,50.141964,48.086319,22.722284,5.692445
1,2017,1,1,2,2.304043,2.304043,2.957614,49.909744,48.165528,22.596884,5.674995
2,2017,1,1,3,2.201951,2.201951,2.964389,50.024066,48.442929,22.698529,5.667357
3,2017,1,1,4,2.125386,2.125386,2.960446,49.957527,48.467322,22.692695,5.601962
4,2017,1,1,5,2.106490,2.106490,2.962206,49.987221,48.258539,22.994024,5.597869
...,...,...,...,...,...,...,...,...,...,...,...
38659,2021,5,31,20,1.509553,1.509553,2.096975,35.386450,30.605213,17.667013,3.540661
38660,2021,5,31,21,1.526050,1.526050,2.053280,34.649102,30.864526,17.374328,3.403950
38661,2021,5,31,22,1.528969,1.528969,2.027104,34.207382,31.353910,16.723888,3.253067
38662,2021,5,31,23,1.531375,1.531375,2.020259,34.091867,32.302614,15.945077,3.159544


In [3]:
building = pd.read_csv('../temp/dataset_03[1층].csv')
building.replace('0', 0, inplace=True)
building

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,공용공간,공용공간,공용공간,0,0,0,0,0,공용공간,공용공간,0,0,0,0,0,공용공간,공용공간,공용공간,0
1,0,연구공간,연구공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,연구공간,연구공간,0
2,0,연구공간,연구공간,공용공간,교수연구,교수연구,강의지원,연구공간,연구공간,연구공간,연구공간,강의지원,교수연구,교수연구,강의지원,행정지원,공용공간,연구공간,연구공간,0
3,0,0,연구공간,공용공간,0,0,0,0,0,0,0,0,0,0,0,0,공용공간,연구공간,0,0
4,0,0,연구공간,공용공간,0,0,0,0,0,0,0,0,0,0,0,0,공용공간,연구공간,0,0
5,0,0,행정지원,공용공간,0,0,0,0,0,공용공간,공용공간,공용공간,0,0,0,0,공용공간,연구공간,연구공간,연구공간
6,학생자율,학생자율,행정지원,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,공용공간,연구공간
7,연구공간,연구공간,연구공간,공용공간,강의지원,강의지원,연구공간,연구공간,교수연구,행정지원,강의지원,강의지원,연구공간,연구공간,학술지원,학생자율,공용공간,연구공간,연구공간,연구공간


In [4]:
img_path = '../dataset/2d/img/elec'
npy_path = '../dataset/2d/npy/elec'

check_path(img_path)
check_path(npy_path)

In [5]:
start = time.time()
# npy save
res = []
for idx in tqdm(elec.index):
    temp_b = building.copy()
    temp_b.replace(cols, elec.iloc[idx][4:], inplace=True)
    res.append(temp_b)
np.save(npy_path + '/2d_elec.npy', np.array(res))
end = time.time()
print('='*10, 'npy', '='*10)
print(f'저장 소요 시간 : {end-start:.2f}')


start = time.time()
max_val = np.max(elec[cols].max())
# png save
for idx in tqdm(elec.index):
    temp_b = building.copy()
    temp_b.replace(cols, elec.iloc[idx][4:], inplace=True)
    img = Image.fromarray((temp_b.to_numpy()/max_val)*255).convert('L')
    img.save(img_path + f'/{int(elec.iloc[idx]["year"])}_{int(elec.iloc[idx]["month"])}_{int(elec.iloc[idx]["day"])}_{int(elec.iloc[idx]["hour"])}.png', 'png')
end = time.time()
print('='*10, 'png', '='*10)
print(f'저장 소요 시간 : {end-start:.2f}')


100%|██████████| 38664/38664 [01:13<00:00, 526.92it/s]


========== npy ==========
저장 소요 시간 : 73.62


100%|██████████| 38664/38664 [01:53<00:00, 341.83it/s]

========== png ==========
저장 소요 시간 : 113.11


In [6]:
print(max_val)

147.0411963816698
